### 애니메이션 만들기

- 분석에 활용될 애니메이션 만들기
- 실패한 패스의 경우 패스의 의도를 직접 레이블링해야하기 때문에 애니메이션 필수

In [1]:
import os, sys
import tqdm
sys.path.append(os.path.abspath(os.path.dirname(os.getcwd())))

In [2]:
import pandas as pd

from datatools.trace_animator import TraceAnimator
from matplotlib import animation

In [3]:
all_events = pd.read_csv(f'../data/unified-data/all-match.csv')
all_events.head()

,team,type,subtype,session,start_frame,start_time,end_frame,end_time,from,to,...,end_x,end_y,phase,game_id,is_left_to_right,goal,ownGoal,eventType,outcome,event_id
0,Away,PASS,PASS,1,1,0.04,3,0.12,B19,B21,...,59.40,30.96,1,1,False,False,False,PASS,True,0
1,Away,PASS,PASS,1,3,0.12,17,0.68,B21,B15,...,62.64,15.12,1,1,False,False,False,PASS,True,1
2,Away,PASS,PASS,1,45,1.80,61,2.44,B15,B19,...,48.60,22.32,1,1,False,False,False,PASS,True,2
3,Away,PASS,PASS,1,77,3.08,96,3.84,B19,B21,...,52.92,33.84,1,1,False,False,False,PASS,True,3
4,Away,PASS,PASS,1,191,7.64,217,8.68,B21,B22,...,34.56,70.56,1,1,False,False,False,PASS,True,4


In [4]:
all_events[all_events["eventType"] == "DRIBBLE"].head(3)

,team,type,subtype,session,start_frame,start_time,end_frame,end_time,from,to,...,end_x,end_y,phase,game_id,is_left_to_right,goal,ownGoal,eventType,outcome,event_id
944,Home,CHALLENGE,DRIBBLE-WON,1,25595,1023.80,25595,1023.80,A01,NaN,...,NaN,NaN,1,1,True,False,False,DRIBBLE,False,944
2071,Away,CHALLENGE,DRIBBLE-WON,2,123079,4923.16,123079,4923.16,B26,NaN,...,NaN,NaN,5,2,False,False,False,DRIBBLE,False,2071
3356,Away,CARRY,CARRY,1,377,15.08,384,15.36,A07,NaN,...,53.676,34.92,1,3,False,False,False,DRIBBLE,True,3356


### 하나의 이벤트 데이터 animation만들기

In [17]:
index = 944

start_frame, end_frame, game_id = all_events.loc[index, ['start_frame', 'end_frame', 'game_id']]

trace = pd.read_csv(f'../data/preprocess-data/tracking-data/match{game_id}.csv')
trace.set_index('frame', inplace=True)

traces = trace[start_frame-100 : end_frame+300]

animator = TraceAnimator(
    trace_dict={"main": traces},
    show_episodes=True,
    show_events=True,
    fps = 25 
)
anim = animator.run()

t0 = traces["time"].iloc[0]
t1 = traces["time"].iloc[-1]
t0_str = f"{int(t0 // 60):02d}.{int(t0 % 60):02d}"
t1_str = f"{int(t1 // 60):02d}.{int(t1 % 60):02d}"

anim_file = f"./{game_id}_{t0_str}-{t1_str}.mp4"
writer = animation.FFMpegWriter(fps=25)
anim.save(anim_file, writer=writer)

### 모든 이벤트 애니메이션 만들기

In [7]:
all_events.loc[245]

team                        Home
type                   BALL LOST
subtype             INTERCEPTION
session                        1
start_frame                36410
start_time                1456.4
end_frame                  36410
end_time                  1456.4
from                         A08
to                           NaN
start_x                    63.72
start_y                     8.64
end_x                        NaN
end_y                        NaN
phase                          1
game_id                        1
is_left_to_right            True
goal                       False
ownGoal                    False
eventType                   PASS
outcome                    False
event_id                     235
Name: 235, dtype: object

In [9]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

game_ids = [1, 2, 3]
all_events = pd.read_csv('../data/unified-data/all-match.csv')

for game_id in game_ids:
    trace = pd.read_csv(f'../data/preprocess-data/tracking-data/match{game_id}.csv')
    trace.set_index('frame', inplace=True)

    events = all_events[(all_events['game_id'] == game_id)]

    for _, row in events.iterrows():
        event_id = row['event_id']

        # 하나의 이벤트 예시로 애니메이션 만들기
        if event_id != 245:
            continue

        start_frame = row['start_frame']
        end_frame = row['end_frame']

        frame_traces = trace.loc[start_frame - 100 : end_frame + 300].reset_index()

        # highlight_time: -100~300frame동안의 영상을 보면 어느 패스를 분석하고 싶은지 따로 강조해줘야함
        # animation은 t가 0부터 시작하기 때문에, tracking-data의 frame, time정보를 그대로 활용할 수 없음 -> 새로운 time_index제작함
        highlight_time = frame_traces[start_frame : end_frame].index.to_list()

        animator = TraceAnimator(
            trace_dict={"main": frame_traces},
            highlight_time = highlight_time,
            show_episodes=True,
            show_events=True,
            fps = 25,
            play_speed=1,
        )
        
        anim = animator.run()

        anim_file = f'../pass-animation/{event_id}.mp4'
        writer = animation.FFMpegWriter(fps=25)
        anim.save(anim_file, writer=writer)